<a href="https://colab.research.google.com/github/hopecris31/Reddit_Transformer/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import transformers
import math
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True, max_length=10)
model = AutoModelForCausalLM.from_pretrained(checkpoint)


In [13]:
from datasets import load_dataset

In [14]:
raw_datasets = load_dataset("reddit", split="train[:1%]")
raw_datasets

Found cached dataset reddit (C:/Users/hopec/.cache/huggingface/datasets/reddit/default/1.0.0/98ba5abea674d3178f7588aa6518a5510dc0c6fa8176d9653a3546d5afcb3969)


Dataset({
    features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
    num_rows: 38483
})

In [15]:
print(raw_datasets.column_names)

['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary']


In [16]:
def tokenize_function(example):
    return tokenizer(example["subreddit"], example["content"])
    

In [17]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=raw_datasets.column_names)
tokenized_datasets

Map (num_proc=4):   0%|          | 0/38483 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [ ]:

tokenized_datasets[1]


In [ ]:
block_size = tokenizer.model_max_length

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum([ex for ex in examples[k] if isinstance(ex, list)], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
lm_datasets

In [ ]:
tokenizer.decode(lm_datasets[1]["input_ids"])

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
model_name = checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-reddit",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH = 'Drive/NLP Final' # [CHANGE ME]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets,
    eval_dataset=lm_datasets,
)

In [ ]:
trainer.train()
drive.mount('/content/drive')
PATH = 'Drive/NLP Final' # [CHANGE ME]
trainer.save_model (PATH + "/ChatGRT")

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")